<h1>Media pose learning</h1>

In [1]:
import cv2
import mediapipe as mp

2024-02-26 19:45:24.724996: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 19:45:24.772887: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 19:45:24.772957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 19:45:24.774355: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 19:45:24.780693: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 19:45:24.781474: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1708956938.734264   12211 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1708956938.741932   12365 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: Mesa Intel(R) UHD Graphics 630 (CFL GT2)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Convert the BGR image to RGB, flip the image around y-axis for correct handedness output
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    
    # To improve performance, optionally mark the image as not writeable to pass by reference.
    image.flags.writeable = False
    results = pose.process(image)

    # Draw the pose annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()

In [5]:
# show live feed with the body landmarks and also a horizontal line in the middle of the frame

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Convert the BGR image to RGB, flip the image around y-axis for correct handedness output
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    
    # To improve performance, optionally mark the image as not writeable to pass by reference.
    image.flags.writeable = False
    results = pose.process(image)

    # Draw the pose annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        h, w, c = image.shape
        cv2.line(image, (0, h//2), (w, h//2), (0, 255, 0), 2)

    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
# show live feed with the body landmarks and also a horizontal line in the middle of shoulder and nose
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Convert the BGR image to RGB, flip the image around y-axis for correct handedness output
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    
    # To improve performance, optionally mark the image as not writeable to pass by reference.
    image.flags.writeable = False
    results = pose.process(image)

    # Draw the pose annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        h, w, c = image.shape
        nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        cv2.line(image, (int(left_shoulder.x*w), int(left_shoulder.y*h)), (int(nose.x*w), int(nose.y*h)), (0, 255, 0), 2)
        cv2.line(image, (int(right_shoulder.x*w), int(right_shoulder.y*h)), (int(nose.x*w), int(nose.y*h)), (0, 255, 0), 2)

    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()

In [10]:
# show live feed with the body landmarks
# also calculate the average horizontal coordinates of the shoulders and use it to draw a horizontal line in between the shoulders and the nose
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Convert the BGR image to RGB, flip the image around y-axis for correct handedness output
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    
    # To improve performance, optionally mark the image as not writeable to pass by reference.
    image.flags.writeable = False
    results = pose.process(image)

    # Draw the pose annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        h, w, c = image.shape
        nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        avg_shoulder_x = (left_shoulder.x + right_shoulder.x) / 2
        # avg_shoulder_y = (left_shoulder.y + right_shoulder.y) / 2
        center_shoulder_nose = (avg_shoulder_x + nose.x) / 2
        cv2.line(image, (0, int(center_shoulder_nose*h)), (w, int(center_shoulder_nose*h)), (0, 255, 0), 2)

    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()

In [12]:
print("nose:", mp_pose.PoseLandmark.NOSE)
print("left shoulder:", mp_pose.PoseLandmark.LEFT_SHOULDER)
print("right shoulder:", mp_pose.PoseLandmark.RIGHT_SHOULDER)



nose: PoseLandmark.NOSE
left shoulder: PoseLandmark.LEFT_SHOULDER
right shoulder: PoseLandmark.RIGHT_SHOULDER


In [3]:
# show live feed with the body landmarks and also a horizontal line in the middle of the frame and if the nose is below the line, make a beep sound
# import winsound
import os
# import beep
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Convert the BGR image to RGB, flip the image around y-axis for correct handedness output
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    
    # To improve performance, optionally mark the image as not writeable to pass by reference.
    image.flags.writeable = False
    results = pose.process(image)

    # Draw the pose annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        h, w, c = image.shape
        cv2.line(image, (0, h//2), (w, h//2), (0, 255, 0), 2)
        nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
        if nose.y > 0.5:
            # winsound.Beep(1000, 200)
            # beep.beep(440, 1000)
            os.system("beep -f 440 -l 1000")

    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()

sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep: not found
sh: 1: beep